In [1]:
import json
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# ✅ 학습된 KoBART 모델 로드
model_checkpoint = "./kobart_best_model"  # 🔹 Best 모델이 저장된 체크포인트 폴더
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint)

In [2]:
# ✅ 모델을 GPU로 이동 (가능하면)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# ✅ 요약 함수 정의 (프롬프트 포함)
def generate_summary(description):
    if not description.strip():
        return "줄거리 없음"  # 🔹 설명이 비어 있으면 줄거리 없음 반환
    
    # ✅ 프롬프트 포함
    prompt = f"다음 문장 중 작가의 연혁과 수상이력은 제외하고 스토리부분만 추출 해주세요. 등장인물의 직업, 스토리 전개, 주요 사건을 포함해주세요. 스토리가 없을 경우 줄거리 없음으로 처리하세요.:\n\n{description}"
    # 🔹 입력 텍스트 토큰화
    input_ids = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).input_ids.to(device)

    # 🔹 모델로 요약 생성
    with torch.no_grad():
        summary_ids = model.generate(
            input_ids, 
            max_length=300,  # ✅ 요약 길이 조정
            min_length=150,  # ✅ 너무 짧은 요약 방지
            num_beams=5,  # ✅ 빔 서치 강화
            early_stopping=True,  # ✅ 조기 종료 (반복 방지)
            temperature=0.7,  # ✅ 단어 선택 다양성 증가
            top_k=50,  # ✅ 확률 높은 50개 단어 중 선택
            top_p=0.9,  # ✅ 확률 분포의 90%를 포함한 단어 선택
            length_penalty=1.5,  # ✅ 긴 문장일수록 더 적절하게 요약하도록 유도
            repetition_penalty=1.5  # ✅ 반복적인 단어 생성을 억제
        )
    
    # 🔹 요약문 디코딩
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary_text

# ✅ 소설 데이터 로드
with open("filtered_novels.json", "r", encoding="utf-8") as f:
    novels_data = json.load(f)

In [7]:
# ✅ 모든 소설의 description을 요약
for novel in novels_data:
    novel["summary"] = generate_summary(novel["description"])

# ✅ 새로운 JSON 파일로 저장
output_file = "All_books_with_summary.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(novels_data, f, ensure_ascii=False, indent=4)

print(f"✅ 요약 완료! 결과가 '{output_file}' 파일에 저장되었습니다.")

✅ 요약 완료! 결과가 'All_books_with_summary.json' 파일에 저장되었습니다.
